In [3]:
import mrp1_data_module as md

import numpy as np
import pandas as pd
import time

In [4]:
scraper = md.Scraper(reddit_api_wrapper="praw")
post_df, comment_df = scraper.query_from_subreddits(query="quantum self:no",
                                                    subreddits=["science", "quantum"],     
                                                    post_limit_per_subreddit=20, 
                                                    sort_by="relevance",
                                                    )


filepath = "S:\\Sync\\University\\2023_MRP_1\\MRP1_WorkDir\\data\\"
#Convert to a standardized object structure before exporting to csv files:
artset = md.ArticleSet.from_reddit_dfs(artset_descript="Test ArticleSet", 
                                    post_df = post_df, 
                                    comment_df = comment_df,
                                    scraper=scraper,)

#print(artset.as_list[0].comments[2])

artset.save_to_csv(filepath)


Searching in subreddit:  science ...
Searching in subreddit:  quantum ...
--- 0.0 seconds --- post processed
--- 0.45889830589294434 seconds --- submission 1 total
--- 0.0 seconds --- post processed
--- 0.3738691806793213 seconds --- submission 2 total
--- 0.0 seconds --- post processed
--- 0.3298947811126709 seconds --- submission 3 total
--- 0.0 seconds --- post processed
--- 0.3251378536224365 seconds --- submission 4 total


KeyboardInterrupt: 

In [ ]:
comment_df["depth"]

from math import modf
math.modf()

In [ ]:
#OR store directly in csv files via a direct mapping:
reddit_postFeatures = ["id","ext_link_url"]
reddit_commentFeatures = ["id", "parent_post_id", "created_utc", "body", "depth","parent_comment_id" , "replies_ids"]

target_articleFeatures = ["article_id", "article_type", "link", "date", "author", "headline", "body", "char_count", "comments_ids"] # [field.name for field in fields(Article)].remove("comments")
target_commentFeatures =  ["comment_id", "article_id", "date", "username", "from_where" ,"body", "level" , "parent_comment_id", "replies_ids"]
direct_target_articleFeatures = target_articleFeatures.copy().remove("article_type", "date", "author", "headline", "body", "char_count", "comments_ids") #Subset of target_articleFeatures which can be directly mapped from the reddit data
direct_target_commentFeatures = target_commentFeatures.copy() #Subset of target_commentFeatures which can be directly mapped from the reddit data

reddit_to_commentdf_map = dict(zip(direct_target_articleFeatures, reddit_postFeatures))
reddit_to_articledf_map = dict(zip(direct_target_commentFeatures, reddit_commentFeatures))

export_article_df = pd.DataFrame(columns=target_articleFeatures)
export_comment_df = pd.DataFrame(columns=target_commentFeatures)
export_article_df.loc[:, reddit_to_articledf_map.keys()] = post_df.loc[:, reddit_to_articledf_map.values()]
export_comment_df.loc[:, reddit_to_commentdf_map.keys()] = comment_df.loc[:, reddit_to_commentdf_map.values()]

export_article_df.loc[:,"article_type"] = "web_article"
#Parse: "date", "author", "headline", "body" from ext link url
# 
export_article_df.loc[:,"char_count"] = len(export_article_df.loc[:,"body"]) 
# Map "comments_ids" to the articles


In [ ]:
from trafilatura import fetch_url, extract, bare_extraction
from courlan import validate_url, check_url, clean_url
import requests

#url = 'https://github.blog/2019-03-29-leader-spotlight-erin-spiceland/'
url = "https://scottaaronson.blog/?p=6871"
url = "https://tweakers.net/nieuws/189740/qutech-start-quantum-network-explorer-voor-publieke-experimenten-quantumnetwerk.html"
url = "https://www.quantamagazine.org/how-quantum-physicists-flipped-time-and-how-they-didnt-20230127/"
#url = "https://news.st-andrews.ac.uk/archive/st-andrews-scientists-turn-up-the-heat-on-physics-phenomenon/#:~:text=A%20'quantum%20harmonic%20oscillator'%20%E2%80%93,the%20University%20of%20St%20Andrews."
#url = "https://www.vice.com/en/article/88qj3z/government-scientists-discover-entirely-new-kind-of-quantum-entanglement-in-breakthrough"
#url = "https://www.nature.com/articles/s41534-022-00631-2"
url = "https://www.google.com/url?sa=t&rct=j&q=&esrc=s&source=web&cd=&cad=rja&uact=8&ved=2ahUKEwiX073B8vH8AhUThP0HHe5FDd8QtwJ6BAgSEAI&url=https%3A%2F%2Fwww.youtube.com%2Fwatch%3Fv%3DjHoEjvuPoB8&usg=AOvVaw2oWlY57R1_JHg0bgTgPkoW"
url = "https://youtu.be/jHoEjvuPoB8"

assert validate_url(url)[0] == True #check whether url can be resolved or is damaged

url = clean_url(url)
downloaded = fetch_url(url)

result = bare_extraction(downloaded, 
                            url=url, 
                            #include_comments=True,
                            #include_formatting=False,
                            favor_precision=True, #favor recall
                            with_metadata=True,
                            #deduplicate=True, 
                            only_with_metadata=True,
                            #url_blacklist=None,
                            as_dict=True,
                            )

#print(result.keys())
print(result["title"], "...", result["author"],"...",  result["date"])
#print(result["description"])
print(result["text"])


#extract_comments()
#sanitize()
#trim()

In [ ]:
from newspaper import Article

a = Article(url)
a.download()
a.parse()
#print(a.summary)
print(a.title, a.authors, a.publish_date)
print("Text: ", a.text)

#print(a.nlp())

In [ ]:
import requests
from readabilipy import simple_json_from_html_string
req = requests.get(url)
article = simple_json_from_html_string(req.text, use_readability=True)
#print(article.keys())
print(article["title"], article["byline"], article["date"] )
print(article["plain_text"])